In [2]:
import networkx as nx
import dgl
import torch
from sklearn.preprocessing import LabelEncoder
import os

# --- Load graph từ GraphML ---
print("🔄 Đang tải đồ thị từ coco_kg.graphml...")
G_nx = nx.read_graphml("coco_kg.graphml")

# --- Encode label của node ---
print("🔧 Đang mã hóa node labels...")
node_labels = []
for _, data in G_nx.nodes(data=True):
    node_labels.append(data.get("label", "unknown"))

node_encoder = LabelEncoder()
encoded_node_labels = node_encoder.fit_transform(node_labels)

# Gán thành thuộc tính 'feature' cho node
for i, (_, data) in enumerate(G_nx.nodes(data=True)):
    data["feature"] = int(encoded_node_labels[i])

# --- Encode edge relation và source ---
print("🔧 Đang mã hóa edge attributes...")
rel_list, src_list = [], []
for _, _, data in G_nx.edges(data=True):
    rel_list.append(data.get("relation", "unknown"))
    src_list.append(data.get("source", "unknown"))

rel_encoder = LabelEncoder()
src_encoder = LabelEncoder()
rel_encoded = rel_encoder.fit_transform(rel_list)
src_encoded = src_encoder.fit_transform(src_list)

# Gán giá trị số lại vào cạnh
for i, (u, v, data) in enumerate(G_nx.edges(data=True)):
    data["relation"] = int(rel_encoded[i])
    data["source"] = int(src_encoded[i])

# --- Chuyển sang DGLGraph ---
print("🔁 Đang chuyển sang DGLGraph...")
G_dgl = dgl.from_networkx(G_nx, node_attrs=["feature"], edge_attrs=["relation", "source"])

# --- Lưu lại DGLGraph và encoders ---
torch.save(G_dgl, "coco_kg_dgl.pt")
torch.save(node_encoder.classes_, "label_classes.pt")
torch.save(rel_encoder.classes_, "relation_classes.pt")
torch.save(src_encoder.classes_, "source_classes.pt")

print("✅ Hoàn tất! Đã lưu DGLGraph và các bộ mã hóa.")

🔄 Đang tải đồ thị từ coco_kg.graphml...
🔧 Đang mã hóa node labels...
🔧 Đang mã hóa edge attributes...
🔁 Đang chuyển sang DGLGraph...
✅ Hoàn tất! Đã lưu DGLGraph và các bộ mã hóa.
